In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from random import seed, random

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

plt.rcParams['figure.figsize'] = [10, 7]
%config InlineBackend.figure_format = 'retina'

In [2]:
data = pd.read_csv("../datasets/heroes/hero-network.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574467 entries, 0 to 574466
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   hero1   574467 non-null  object
 1   hero2   574467 non-null  object
dtypes: object(2)
memory usage: 8.8+ MB


In [4]:
data.head()

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL


## Solving Max-Cut problem classically

In [5]:
seed(None)

In [23]:
heroes = [
 'MEDUSA/MEDUSALITH AM',
 'ARCLIGHT/PHILLIPA SO',
 'WOLVERINE/LOGAN ',
 'SCARLET WITCH/WANDA ',
 'ARAGORN',
 'OVERMIND/GROM',
 'BATTLEAXE/',
 'ION/',
 'PINK PEARL/'
]

In [24]:
relations = data[(data.hero1.isin(heroes)) & (data.hero2.isin(heroes))]

In [25]:
def maxcut(assigned_nodes, edges):
    return sum(edges.apply(lambda edge: (1 - assigned_nodes[edge['hero1']] *assigned_nodes[edge['hero2']])/2, axis=1))

In [26]:
single_group = { name: -1 for name in heroes }
single_group

{'MEDUSA/MEDUSALITH AM': -1,
 'ARCLIGHT/PHILLIPA SO': -1,
 'WOLVERINE/LOGAN ': -1,
 'SCARLET WITCH/WANDA ': -1,
 'ARAGORN': -1,
 'OVERMIND/GROM': -1,
 'BATTLEAXE/': -1,
 'ION/': -1,
 'PINK PEARL/': -1}

In [27]:
edges = relations
edges['check_string'] = edges.apply(lambda row: ''.join(sorted([row['hero1'], row['hero2']])), axis=1)

In [28]:
edges = edges.drop_duplicates('check_string')
edges.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 895 to 390162
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   hero1         13 non-null     object
 1   hero2         13 non-null     object
 2   check_string  13 non-null     object
dtypes: object(3)
memory usage: 416.0+ bytes


In [37]:
edges.head()

,hero1,hero2,check_string
895,SCARLET WITCH/WANDA,WOLVERINE/LOGAN,SCARLET WITCH/WANDA WOLVERINE/LOGAN
33894,MEDUSA/MEDUSALITH AM,WOLVERINE/LOGAN,MEDUSA/MEDUSALITH AMWOLVERINE/LOGAN
34882,ARAGORN,SCARLET WITCH/WANDA,ARAGORNSCARLET WITCH/WANDA
34992,SCARLET WITCH/WANDA,MEDUSA/MEDUSALITH AM,MEDUSA/MEDUSALITH AMSCARLET WITCH/WANDA
42019,SCARLET WITCH/WANDA,OVERMIND/GROM,OVERMIND/GROMSCARLET WITCH/WANDA


In [29]:
maxcut(single_group, edges)

0.0

In [30]:
random_groups = {name: -1 if round(random()) == 0 else 1 for name in heroes}

In [31]:
random_groups

{'MEDUSA/MEDUSALITH AM': 1,
 'ARCLIGHT/PHILLIPA SO': 1,
 'WOLVERINE/LOGAN ': 1,
 'SCARLET WITCH/WANDA ': 1,
 'ARAGORN': -1,
 'OVERMIND/GROM': -1,
 'BATTLEAXE/': 1,
 'ION/': -1,
 'PINK PEARL/': -1}

In [32]:
maxcut(random_groups, edges)

6.0

In [33]:
# brute force
def search(group, open_heroes):
    best = (0, None)
    
    if len(open_heroes) > 0:
        new = dict(group)
        new[open_heroes[0]] = new[open_heroes[0]]*(-1)

        result1 = search(group, open_heroes[1:])
        result2 = search(new, open_heroes[1:])
    
        best = result1 if result1[0] > result2[0] else result2
    else:
        return (maxcut(group, edges), group)
    
    return best

In [34]:
search(single_group, heroes)

(9.0,
 {'MEDUSA/MEDUSALITH AM': 1,
  'ARCLIGHT/PHILLIPA SO': 1,
  'WOLVERINE/LOGAN ': -1,
  'SCARLET WITCH/WANDA ': 1,
  'ARAGORN': 1,
  'OVERMIND/GROM': -1,
  'BATTLEAXE/': 1,
  'ION/': -1,
  'PINK PEARL/': -1})